**事前準備**

1. [e-Stat](https://www.e-stat.go.jp/gis/statmap-search?page=1&type=2&aggregateUnitForBoundary=A&toukeiCode=00200521&toukeiYear=2015&serveyId=A002005212015&prefCode=13&coordsys=1&format=kml) から東京都全域の境界データをダウンロード（.zipファイル）
2. .zipファイルを解凍し,.kmzファイルを取得

**八日京周辺エリアの境界ポリゴンデータの取り出し**

<!-- 
区名
- 千代田区
- 中央区
- 港区 -->

In [1]:
import os
import folium
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
# .kmzファイルから.kmlデータを取り出す
with zipfile.ZipFile('r2ka13.kmz','r') as post:
    for info in post.infolist():
        doc_kml = post.read(info.filename)

In [3]:
# kmlデータから全都市情報を抜き出す
soup = BeautifulSoup(doc_kml, 'html.parser')
city_data = soup.find_all('placemark')

In [45]:
# エリアを格納するデータフレーム
DF_CITY = pd.DataFrame()

# 上記をkeyとするポリゴン配列を保存する辞書
POLYGON_DICT={}

# 各エリア毎にエリア名・ポリゴンデータを抽出
for i,data in tqdm(enumerate(city_data),total=len(city_data)):
    
    # 全項目を取得
    attr_dic={}
    attr_dic["ID"] = i
    for d in data.find_all("simpledata"):
        attr = d.attrs.get('name')
        if len(d.contents)>0:
            attr_dic[f"{attr}"] = d.contents[0]
        else:
            attr_dic[f"{attr}"] = np.nan
            
    # データフレームにエリア名を格納
    DF_CITY = DF_CITY.append(attr_dic,ignore_index=True)
        
    # ポリゴン情報を取り出す
    polygon = data.find("coordinates").contents[0].replace(" ","").split("\n")[1:-1]
    
    # ポリゴン情報を格納する
    POLYGON_DICT[i] = [[float(e) for e in ele.split(",")[:-1][::-1]] for ele in polygon]
    
# DF_CITY = DF_CITY.reset_index().rename(columns={"index":"ID"})

100%|██████████| 6021/6021 [01:46<00:00, 56.59it/s] 


In [19]:
# DF_CITY.isnull().sum()
# DF_CITY.shape
# DF_CITY.columns

In [63]:
# 分析対象エリア（八日京エリア周辺）の取得
df_area = DF_CITY[DF_CITY["CITY_NAME"].isin(['千代田区', '中央区', '港区'])]

In [64]:
df_area.head(1)

,AREA,AREA_MAX_F,CITY,CITY_NAME,DUMMY1,HCODE,ID,JINKO,KBSUM,KCODE1,...,PERIMETER,PREF,PREF_NAME,R2KAxx,R2KAxx_ID,SETAI,S_AREA,S_NAME,X_CODE,Y_CODE
5,379196,M,101,千代田区,-,8101,5.0,10,11,0010-01,...,3248.984,13,東京都,1,0,10,001001,丸の内一丁目,139.765963,35.682153


In [80]:
# import re

# # エリア名を取得
# area_all = df_area["S_NAME"].unique()

# # n丁目を省略
# area = [re.sub('(一|二|三|四|五|六|七|八|九)丁目', '', s) for s in area_all]
# # 神田エリアを統合
# area = [re.sub('(.*)神田(.*)','神田',s) for s in area]
# # 日本橋エリアを統合
# area = [re.sub('(.*)日本橋(.*)','日本橋',s) for s in area]

In [82]:
# 地図の定義
japan_map = folium.Map(
    location=[35.6809591,139.7673068],
    zoom_start=14,
    #tiles='cartodbpositron',#'Stamen Terrain'
)

# 各エリアの描画
for i in range(len(df_area)):
    
    # エリア情報の取り出し
    data = df_area.iloc[i]
    idx = data["ID"]
    area_name = data["CITY_NAME"]+" "+data["S_NAME"]
    longitude = data["X_CODE"]
    latitude = data["Y_CODE"]
    
    # color = color_list[]
    
    folium.Polygon(
        locations=POLYGON_DICT[idx], 
        weight=2, color='red',fill=True, fill_opacity=0.4,
        popup=area_name,
        ).add_to(japan_map)

In [83]:
japan_map

In [ ]:
# !pip install turfpy
# https://qiita.com/Bashi50/items/b9995d430f22ce42eac9

# エリア内の存在判定
# from turfpy.measurement import boolean_point_in_polygon
# from geojson import Point, Polygon, Feature

# point = Feature(geometry=Point((0.5, 1.1)))
# polygon = Polygon(
#     [
#         [
#             (0.0, 0.0),
#             (1.0, 2.0),
#             (3.0, 2.0),
#             (2.0, 0.0)
#         ]
#     ]
# )

# if boolean_point_in_polygon(point, polygon):
#     print('領域内')
# else:
#     print('領域外')

In [ ]:
# バス路線
# https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-N07.html#prefecture13

# 高速道路
# https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-N06-v2_0.html

# バス停留所
# https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-P11.html

# 鉄道
# https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-N02-v3_0.html